In [ ]:
import os
import re
import openai
import sqlite3
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class Config:
    """Manages configurable parameters."""
    def __init__(self, directory, topic, api_key, model_name='all-MiniLM-L6-v2', llm_models=['gpt-4'], db_path='results.db'):
        self.directory = directory
        self.topic = topic
        self.api_key = api_key
        self.model_name = model_name
        self.llm_models = llm_models if isinstance(llm_models, list) else [llm_models]
        self.db_path = db_path
        openai.api_key = api_key

class MarkdownCleaner:
    @staticmethod
    def clean_markdown(md_text):
        """Remove Markdown syntax and extract clean text."""
        md_text = re.sub(r'\[.*?\]\(.*?\)', '', md_text)  # Remove links
        md_text = re.sub(r'#{1,6}\s*', '', md_text)  # Remove headers
        md_text = re.sub(r'(```.*?```|`.*?`)', '', md_text, flags=re.DOTALL)  # Remove code blocks
        md_text = re.sub(r'\*{1,2}|\_{1,2}', '', md_text)  # Remove bold/italic formatting
        md_text = re.sub(r'>\s*', '', md_text)  # Remove blockquotes
        md_text = re.sub(r'[-+*]\s+', '', md_text)  # Remove bullet points
        md_text = re.sub(r'\d+\.\s+', '', md_text)  # Remove numbered lists
        return md_text.strip()

class DocumentEvaluator:
    def __init__(self, config):
        self.config = config
        self.model = SentenceTransformer(config.model_name)
        self._initialize_database()
    
    def _initialize_database(self):
        """Initialize the database table to store results."""
        conn = sqlite3.connect(self.config.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS document_scores (
                filename TEXT PRIMARY KEY,
                relevance_score REAL,
                correctness_score REAL,
                final_score REAL
            )
        ''')
        conn.commit()
        conn.close()
    
    def save_results_to_db(self, rankings):
        """Save ranked document results to the database."""
        conn = sqlite3.connect(self.config.db_path)
        cursor = conn.cursor()
        
        for filename, relevance, correctness, final_score in rankings:
            cursor.execute('''
                INSERT INTO document_scores (filename, relevance_score, correctness_score, final_score)
                VALUES (?, ?, ?, ?)
                ON CONFLICT(filename) DO UPDATE SET
                    relevance_score=excluded.relevance_score,
                    correctness_score=excluded.correctness_score,
                    final_score=excluded.final_score
            ''', (filename, relevance, correctness, final_score))
        
        conn.commit()
        conn.close()
    
    def load_markdown_files(self):
        """Load all Markdown files from a directory and extract clean text."""
        documents = []
        for filename in os.listdir(self.config.directory):
            if filename.endswith(".md"):
                with open(os.path.join(self.config.directory, filename), "r", encoding="utf-8") as file:
                    text = file.read()
                    clean_text = MarkdownCleaner.clean_markdown(text)
                    documents.append((filename, clean_text))
        return documents
    
    def compute_relevance(self, documents):
        """Compute relevance scores using embeddings."""
        doc_texts = [doc[1] for doc in documents]
        doc_embeddings = self.model.encode(doc_texts)
        topic_embedding = self.model.encode([self.config.topic])[0]  # Topic embedding

        # Compute similarity
        scores = cosine_similarity([topic_embedding], doc_embeddings)[0]

        # Attach scores to documents
        relevance_scores = {doc[0]: score for doc, score in zip(documents, scores)}
        return relevance_scores
    
    def evaluate_correctness_with_llm(self, document_text):
        """Use multiple LLMs to evaluate correctness based on factual accuracy and logical consistency."""
        total_score = 0
        for model in self.config.llm_models:
            prompt = f"""
            You are an expert reviewer. Evaluate the following document for correctness based on:
            - Factual accuracy
            - Logical consistency
            - Clarity and grammar
            
            Provide a score from 1 to 10 and justify your score.

            Document:
            ```
            {document_text}
            ```
            """
            response = openai.ChatCompletion.create(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            
            result = response["choices"][0]["message"]["content"]
            score_match = re.search(r'Correctness Score:\s*(\d+)/10', result)
            score = int(score_match.group(1)) if score_match else 5
            total_score += score
        
        final_score = total_score / len(self.config.llm_models)
        return final_score
    
    def rank_documents(self):
        """Rank documents based on relevance and correctness."""
        documents = self.load_markdown_files()
        relevance_scores = self.compute_relevance(documents)
        correctness_scores = {}
        
        for filename, text in documents:
            score = self.evaluate_correctness_with_llm(text)
            correctness_scores[filename] = score
        
        rankings = []
        for filename in relevance_scores.keys():
            relevance = relevance_scores[filename]
            correctness = correctness_scores[filename]
            final_score = (relevance * 5) + (correctness * 5)
            rankings.append((filename, relevance, correctness, final_score))
        
        rankings.sort(key=lambda x: x[3], reverse=True)
        self.save_results_to_db(rankings)
        
        df = pd.DataFrame(rankings, columns=["File", "Relevance Score", "Correctness Score", "Final Score"])
        import ace_tools as tools
        tools.display_dataframe_to_user(name="Ranked Documents", dataframe=df)

# Example Usage
config = Config(directory="path_to_markdown_files", 
                topic="Artificial Intelligence and Deep Learning", 
                api_key="your-api-key", 
                llm_models=["gpt-4", "gpt-3.5-turbo"],
                db_path="results.db")

evaluator = DocumentEvaluator(config)
evaluator.rank_documents()
